# Laboratorio 9

## Integrantes

### Sergio Orellana - 221122

### Andre Marroquin - 22266

### Rodrigo Mansilla - 22611

# Link del repositorio

https://github.com/mar22266/LABORATORIOS-IA.git


# Tasks 1 - Teoría

## 1. ¿Cuál es la diferencia entre Modelos de Markov y Hidden Markov Models?

        R// Un Modelo de Markov asume que los estados del sistema son directamente observables y que la probabilidad de pasar al siguiente estado depende únicamente del estado actual, lo que es conocido como la propiedad de Markov. Por otro lado, un Hidden Markov Model esconde esos estados internos, ya que solo se pueden observar resultados o señales que provienen de un estado interno oculto que no es posible medir directamente. 


## 2. ¿Qué son los Factorial HMM (Hidden Markov Models)?
        R// Un Factorial HMM se puede decir que es una versión más avanzada del modelo oculto de Markov que divide el estado oculto en varias partes independientes, cada una con su propia cadena de Markov. Lo que hace permitir que este logre representar situaciones donde varios procesos ocurren al mismo tiempo, como señales que se combinan entre sí. Pero, como el número de posibles combinaciones de estados aumenta muy rápido al agregar más cadenas, es oor eso que la mayormente se utilizan métodos aproximados, como los enfoques variacionales, para poder realizar los cálculos de manera eficiente.


## 3. Especifique en sus propias palabras el algoritmo Forward-Backward para HMM
        R// El algoritmo Forward-Backward se usa en los modelos ocultos de Markov para saber qué tan probable es que el sistema esté en cierto estado en un momento específico, tomando en cuenta todas las observaciones disponibles. Funciona en dos partes primero, la parte forward que va sumando las probabilidades desde el inicio hasta ese momento, y luego la parte backward calcula qué tan probable es que, desde ese estado, se generen las observaciones restantes. Al juntar las 2 partes, se obtiene una buena estimación de en qué estado estaba el sistema en cada instante.



## 4. ¿Por qué es necesario el paso de Backward en el algoritmo Forward-Backward?
        R// Sin backward, solo se tendría en cuenta la información acumulada hasta el instante actual, lo que limita la precisión de la estimación del estado. Al incluir la pasada backward, se incorpora también la información de las observaciones futuras, lo que permite mejorar la estimación del estado en cada momento. De esta forma, al combinar las pasadas forward y backward, se logra una probabilidad más precisa de cada estado, ya que se considera tanto la historia pasada como la futura, lo cual es bueno para obtener un resultado más completo y exacto.



# Task 2 - Algoritmo Forward Backward en HMM 

In [ ]:
import random

class HMM:
    def __init__(self, states, observations, initial_prob, transition_prob, emission_prob):
        # Inicializar parametros de HMM
        self.states = states
        self.observations = observations
        self.initial_prob = initial_prob
        self.transition_prob = transition_prob
        self.emission_prob = emission_prob

    # Genera una secuencia de observaciones  basada en el modelo
    def generate_sequence(self, length):
        sequence = []
        current_state = random.choices(
            self.states,
            weights=[self.initial_prob[s] for s in self.states]
        )[0]
        sequence.append(
            random.choices(
                self.observations,
                weights=[self.emission_prob[current_state][o] for o in self.observations]
            )[0]
        )
        for _ in range(1, length):
            current_state = random.choices(
                self.states,
                weights=[self.transition_prob[current_state][s] for s in self.states]
            )[0]
            sequence.append(
                random.choices(
                    self.observations,
                    weights=[self.emission_prob[current_state][o] for o in self.observations]
                )[0]
            )
        return sequence
    # Calcula las probabilidades hacia adelante alfa para una secuencia de observaciones
    def forward(self, observations):
        T = len(observations)
        alpha = [dict() for _ in range(T)]
        for s in self.states:
            alpha[0][s] = self.initial_prob[s] * self.emission_prob[s][observations[0]]
        for t in range(1, T):
            for s in self.states:
                alpha[t][s] = self.emission_prob[s][observations[t]] * sum(
                    alpha[t - 1][sp] * self.transition_prob[sp][s]
                    for sp in self.states
                )
        return alpha
    # Calcula las probabilidades hacia atrás beta para una secuencia de observaciones
    def backward(self, observations):
        T = len(observations)
        beta = [dict() for _ in range(T)]
        for s in self.states:
            beta[T - 1][s] = 1.0
        for t in range(T - 2, -1, -1):
            for s in self.states:
                beta[t][s] = sum(
                    self.transition_prob[s][sp] *
                    self.emission_prob[sp][observations[t + 1]] *
                    beta[t + 1][sp]
                    for sp in self.states
                )
        return beta
    # Combina alpha y beta para calcular probabilidades de estado gamma en cada paso de tiempo
    def compute_state_probabilities(self, observations):
        alpha = self.forward(observations)
        beta = self.backward(observations)
        T = len(observations)
        gamma = [dict() for _ in range(T)]
        for t in range(T):
            denom = sum(alpha[t][s] * beta[t][s] for s in self.states)
            for s in self.states:
                gamma[t][s] = (alpha[t][s] * beta[t][s]) / denom if denom != 0 else 0
        return gamma



In [ ]:
# Definir parametros del HMM
states = ['Sunny', 'Rainy']
observations = ['Sunny', 'Rainy']
initial_prob = {'Sunny': 0.5, 'Rainy': 0.5}
transition_prob = {
    'Sunny': {'Sunny': 0.8, 'Rainy': 0.2},
    'Rainy': {'Sunny': 0.4, 'Rainy': 0.6}
}
emission_prob = {
    'Sunny': {'Sunny': 0.8, 'Rainy': 0.2},
    'Rainy': {'Sunny': 0.3, 'Rainy': 0.7}
}

# Crear instancia de HMM
hmm = HMM(states, observations, initial_prob, transition_prob, emission_prob)

# Generar secuencia de observaciones 
seq = hmm.generate_sequence(5)
print("Secuencia Generada:", seq)

# Secuencia de observaciones para algoritmos
obs_seq = ['Sunny', 'Sunny', 'Rainy']

# Probabilidades Forward
fwd = hmm.forward(obs_seq)
print("\nProbabilidades Forward:")
for t, probs in enumerate(fwd):
    print(f" t={t}: {probs}")

# Probabilidades Backward
bwd = hmm.backward(obs_seq)
print("\nProbabilidades Backward:")
for t, probs in enumerate(bwd):
    print(f" t={t}: {probs}")

# Probabilidades de Estado 
gamma = hmm.compute_state_probabilities(obs_seq)
print("\nProbabilidades de Estado:")
for t, probs in enumerate(gamma):
    print(f" t={t}: {probs}")


Secuencia Generada: ['Sunny', 'Rainy', 'Sunny', 'Rainy', 'Rainy']

Probabilidades Forward:
 t=0: {'Sunny': 0.4, 'Rainy': 0.15}
 t=1: {'Sunny': 0.30400000000000005, 'Rainy': 0.051000000000000004}
 t=2: {'Sunny': 0.05272000000000002, 'Rainy': 0.06398000000000001}

Probabilidades Backward:
 t=0: {'Sunny': 0.22200000000000006, 'Rainy': 0.18600000000000003}
 t=1: {'Sunny': 0.30000000000000004, 'Rainy': 0.5}
 t=2: {'Sunny': 1.0, 'Rainy': 1.0}

Probabilidades de Estado:
 t=0: {'Sunny': 0.7609254498714653, 'Rainy': 0.23907455012853465}
 t=1: {'Sunny': 0.7814910025706941, 'Rainy': 0.21850899742930588}
 t=2: {'Sunny': 0.45175664095972584, 'Rainy': 0.5482433590402742}


Refrencias:

- GeeksforGeeks. (2024b, junio 25). What Is the Difference Between Markov Chains and Hidden Markov Models? GeeksforGeeks. https://www.geeksforgeeks.org/what-is-the-difference-between-markov-chains-and-hidden-markov-models/

- Genç, E. (2024, 29 febrero). Factorial Hidden Markov Model for Time Series Analysis in Python. Medium. https://medium.com/@erdal.genc09/factorial-hidden-markov-model-for-time-series-analysis-in-python-4c6d6f33860b

- Forward and Backward Algorithm in Hidden Markov Model. (2019, 17 febrero). A Developer Diary. https://adeveloperdiary.com/data-science/machine-learning/forward-and-backward-algorithm-in-hidden-markov-model/

- Forward-backward algorithm for HMM. (s. f.). Cross Validated. https://stats.stackexchange.com/questions/275413/forward-backward-algorithm-for-hmm

- Pramod, O. (2023, 30 diciembre). Hidden Markov Models: The Secret Sauce in Natural Language Processing. Medium. https://medium.com/@ompramod9921/hidden-markov-models-the-secret-sauce-in-natural-language-processing-5273503a33f6
